### Cell 1: Imports & Setup

In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data

# Enable renderer
alt.renderers.enable('default')

print("✅ Libraries imported.")

✅ Libraries imported.


### Cell 2: Data Loading & Feature Engineering

In [2]:
try:
    df = pd.read_csv('country_economics_data.csv')
except FileNotFoundError:
    print("⚠️ CSV not found. Please ensure 'country_economics_data.csv' is in the directory.")
    df = pd.DataFrame(columns=['Name', 'ID', 'Region', 'Inflation Rate', 'Jobless Rate', 'Interest Rate', 'Latitude'])

# 1. Clean Data
# We need Inflation, Jobless Rate, and Interest Rate for this dashboard
df_clean = df.dropna(subset=['Inflation Rate', 'Jobless Rate', 'Interest Rate', 'ID', 'Latitude']).copy()

# 2. Feature Engineering

# The Misery Index = Inflation + Unemployment
df_clean['Misery_Index'] = df_clean['Inflation Rate'] + df_clean['Jobless Rate']

# Hemisphere (for B&W Shape Distinction)
# If Latitude >= 0, it's North. Otherwise South.
df_clean['Hemisphere'] = df_clean['Latitude'].apply(lambda x: 'North' if x >= 0 else 'South')

# Ensure ID is int for map
df_clean['ID'] = df_clean['ID'].astype(int)

print("✅ Features created: Misery_Index, Hemisphere")
print(df_clean[['Name', 'Misery_Index', 'Hemisphere', 'Inflation Rate', 'Jobless Rate']].head())

✅ Features created: Misery_Index, Hemisphere
          Name  Misery_Index Hemisphere  Inflation Rate  Jobless Rate
0  Afghanistan         13.50      North            0.20         13.30
1      Albania         11.20      North            2.50          8.70
2      Algeria         11.21      North           -0.22         11.43
3       Angola         48.88      South           19.48         29.40
4    Argentina         44.50      South           36.60          7.90


### Cell 3: Interaction Setup

In [3]:
# Interaction: Brush to highlight "Crisis" clusters on the Scatter plot
brush = alt.selection_interval()

print("✅ Interaction defined.")

✅ Interaction defined.


### Cell 4: View 1 - The Misery Map (Main View)

In [8]:
world_map = alt.topo_feature(data.world_110m.url, 'countries')

map_view = alt.Chart(world_map).mark_geoshape(
    stroke='white',
    strokeWidth=0.5
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df_clean, 'ID', ['Name', 'Misery_Index', 'Inflation Rate', 'Jobless Rate'])
).encode(
    color=alt.Color(
        'Misery_Index:Q',
        scale=alt.Scale(scheme='orangered'),
        title='Misery Index (Inflation + Jobless)'
    ),
    tooltip=[
        alt.Tooltip('Name:N', title='Country'),
        alt.Tooltip('Misery_Index:Q', format='.1f', title='Misery Index'),
        alt.Tooltip('Inflation Rate:Q', format='.1f', title='Inflation (%)'),
        alt.Tooltip('Jobless Rate:Q', format='.1f', title='Unemployment (%)')
    ],
    opacity=alt.condition(brush, alt.value(1), alt.value(0.1))
).project(
    type='equalEarth'
).properties(
    width=1150,  # <--- CHANGED: Full Width
    height=500,  # <--- CHANGED: Top Half Height
    title='Global Economic Distress (The Misery Index)'
).add_params(
    brush
)

map_view

alt.Chart(...)

### Cell 5: View 2 - The Phillips Curve (Scatter Plot)

In [9]:
# 1. The Scatter Points
points = alt.Chart(df_clean).mark_point(filled=True, size=80, opacity=0.7).encode(
    x=alt.X('Jobless Rate:Q', title='Unemployment Rate (%)'),
    y=alt.Y('Inflation Rate:Q', title='Inflation Rate (%)'),
    color=alt.Color(
        'Misery_Index:Q',
        scale=alt.Scale(scheme='orangered'),
        legend=None
    ),
    shape=alt.Shape(
        'Hemisphere:N',
        scale=alt.Scale(domain=['North', 'South'], range=['circle', 'square']),
        title='Hemisphere'
    ),
    tooltip=['Name', 'Inflation Rate', 'Jobless Rate', 'Hemisphere']
).add_params(
    brush
)

# 2. Text Annotation
text_data = pd.DataFrame({'x': [25], 'y': [40], 'label': ['CRISIS CORNER: High Inflation + Joblessness']})

annotation = alt.Chart(text_data).mark_text(
    align='left', baseline='top', dx=5, color='black', fontWeight='bold'
).encode(
    x='x:Q',
    y='y:Q',
    text='label'
)

scatter_view = (points + annotation).properties(
    width=550,   # <--- CHANGED: Fits half width
    height=350,
    title='The Phillips Curve: Inflation vs. Unemployment'
)

scatter_view

alt.LayerChart(...)

### Cell 6: View 3 - Interest Rate Response (Top 10)

In [10]:
bar_view = alt.Chart(df_clean).mark_bar().encode(
    x=alt.X('Interest Rate:Q', title='Interest Rate (%)'),
    y=alt.Y('Name:N', sort='-x', title=None),
    color=alt.value('darkred'),
    tooltip=['Name', 'Interest Rate', 'Inflation Rate']
).transform_filter(
    brush
).transform_window(
    rank='rank()',
    sort=[alt.SortField('Interest Rate', order='descending')]
).transform_filter(
    alt.datum.rank <= 10
).properties(
    width=550,   # <--- CHANGED: Fits half width
    height=350,
    title='Top 10 Interest Rates (Central Bank Response)'
)

bar_view

alt.Chart(...)

### Cell 7: Final Dashboard Assembly

In [11]:
# Bottom Row: Scatter + Bar side by side
bottom_row = alt.hconcat(scatter_view, bar_view, spacing=50)

# Full Dashboard: Map on top of Bottom Row
dashboard = alt.vconcat(map_view, bottom_row, spacing=30).properties(
    title={
        "text": ["Economic Stability Dashboard"],
        "subtitle": ["Highlight the 'Crisis Corner' on the Scatter Plot to locate struggling economies."],
        "fontSize": 24,
        "anchor": "middle"
    }
).configure_view(
    stroke=None
)

dashboard.save('stability_dashboard_full.html')
print("✅ Dashboard saved: 'stability_dashboard_full.html'")

dashboard

C:\Users\Cameron\OneDrive\Desktop\Stage 3\VisualizingEconomicCountryData\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3699: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issues/3891
  exec(code_obj, self.user_global_ns, self.user_ns)


✅ Dashboard saved: 'stability_dashboard_full.html'


alt.VConcatChart(...)